In [ ]:
# Project - validation data to be given at the end of the course & data not to be shared
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.inspection import DecisionBoundaryDisplay

## EDA on Unfiltered DropSeq HCC and MCF data

In [ ]:
df_MCF7_unfiltered = pd.read_csv("DropSeq/MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter=' ').T
print("Dataframe dimensions:", np.shape(df_MCF7_unfiltered))
df_MCF7_unfiltered.head()

In [ ]:
df_MCF7_unfiltered.info()
df_MCF7_unfiltered.describe()

In [ ]:
df_HCC_unfiltered = pd.read_csv("DropSeq/HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter=' ').T
print("Dataframe dimensions:", np.shape(df_HCC_unfiltered))
df_HCC_unfiltered.head()

In [ ]:
df_HCC_unfiltered.info()
df_HCC_unfiltered.describe()

In [ ]:
# Get sets of gene names (column names)
genes_1 = set(df_MCF7_unfiltered.columns)
genes_2 = set(df_HCC_unfiltered.columns)

# Genes only in grouped_sums_1
unique_to_1 = genes_1 - genes_2

# Genes only in grouped_sums_2
unique_to_2 = genes_2 - genes_1

# Union of all unique genes (not shared)
not_in_common = unique_to_1.union(unique_to_2)

# Output
print(f"Genes only in MCF7 dataset: {len(unique_to_1)}")
print(f"Genes only in HCC dataset: {len(unique_to_2)}")
print(f"Total genes not in common: {len(not_in_common)}")
print(f"{not_in_common}")

### MISSING VALUES, DUPLICATES and SPARSITY ANALYSIS 

In [ ]:
# checking for duplicates and missing values
# Percentage of missing values for the whole dataset
print("MCF7 dataset")
total_cells = (df_MCF7_unfiltered.shape[0] * df_MCF7_unfiltered.shape[1])
total_missing = df_MCF7_unfiltered.isnull().sum().sum()
missing_percentage_total = (total_missing / total_cells) * 100
print(f"Total missing values in MCF7 dataset: {missing_percentage_total:.2f}%")

# Missing values per column (only where missing values exist)
missing_per_column = df_MCF7_unfiltered.isnull().sum()

print("\nColumns with missing values (percentage):")
print(missing_per_column)

print(f"count of duplicates: {df_MCF7_unfiltered.duplicated().sum()}")
# Remove if needed
df = df_MCF7_unfiltered.drop_duplicates()

print("\nHCC dataset")
# checking for duplicates and missing values
# Percentage of missing values for the whole dataset
total_cells = (df_HCC_unfiltered.shape[0] * df_HCC_unfiltered.shape[1])
total_missing = df_HCC_unfiltered.isnull().sum().sum()
missing_percentage_total = (total_missing / total_cells) * 100
print(f"Total missing values in HCC dataset: {missing_percentage_total:.2f}%")

# Missing values per column (only where missing values exist)
missing_per_column = df_HCC_unfiltered.isnull().sum()

print("\nColumns with missing values (percentage):")
print(missing_per_column)

print(f"count of duplicates: {df_HCC_unfiltered.duplicated().sum()}")
# Remove if needed
df = df_HCC_unfiltered.drop_duplicates()

In [ ]:
print("MCF7 dataset")
# analyse the transposed dataset with samples as columns and genes as rows
print(f"count of duplicates: {df_MCF7_unfiltered.T.duplicated().sum()}")

# Find duplicated rows
duplicated_rows = df_MCF7_unfiltered.T[df_MCF7_unfiltered.T.duplicated(keep=False)]

# Print the sample names (index labels) of duplicated rows
if not duplicated_rows.empty:
    print("Duplicated Columns of Genes found:")
    print(duplicated_rows.index.tolist())
else:
    print("No duplicated samples found.")

print("\n HCC dataset")
# analyse the transposed dataset with samples as columns and genes as rows
print(f"count of duplicates: {df_HCC_unfiltered.T.duplicated().sum()}")

# Find duplicated rows
duplicated_rows2 = df_HCC_unfiltered.T[df_HCC_unfiltered.T.duplicated(keep=False)]

# Print the sample names (index labels) of duplicated rows
if not duplicated_rows2.empty:
    print("Duplicated Columns of Genes found:")
    print(duplicated_rows2.index.tolist())
else:
    print("No duplicated samples found.")

#### Although all samples are not identiccally zero for all genes (no samples) are redundant, the same cannot be said for the above genes which aren't expressed for any sample. Thus, if one considered only this dataset, the latter should be removed since they increase dimensionality without any additional info, but since the latter genes might not be identically zero for the other datasets, they will be kept for now

### SPARSITY and GENE EXPRESSION by gene's expression mean

In [ ]:
# understanding the sparsity and distribution of the dataset 
non_sparse_genes = [i for i in df_MCF7_unfiltered.columns if df_MCF7_unfiltered[i].mean() >= 1] # non-sparse genes have mean greater than 1 considering also std
non_sparse_genes = sorted(non_sparse_genes, key = lambda x: df_MCF7_unfiltered[x].mean(), reverse=True) # sort according to sparsity and use then for visual
print(f"Number of non-sparse genes in MCF7 dataset: {len(non_sparse_genes)} or {len(non_sparse_genes)/len(df_MCF7_unfiltered.columns):.4}% of all genes")

# understanding the sparsity and distribution of the dataset 
non_sparse_genes2 = [i for i in df_HCC_unfiltered.columns if df_HCC_unfiltered[i].mean() >= 1] # non-sparse genes have mean greater than 1 considering also std
non_sparse_genes2 = sorted(non_sparse_genes2, key = lambda x: df_HCC_unfiltered[x].mean(), reverse=True) # sort according to sparsity and use then for visual
print(f"Number of non-sparse genes in HCC dataset: {len(non_sparse_genes2)} or {len(non_sparse_genes2)/len(df_HCC_unfiltered.columns):.4}% of all genes")

In [ ]:
n = 10
print(f"TOP non-sparse genes in MCF7 dataset: {non_sparse_genes[:n]}")
print(f"TOP non-sparse genes in HCC dataset: {non_sparse_genes2[:n]}")

In [ ]:
fig, ax = plt.subplots(ncols = int(n/2), nrows = 2, figsize = (30,15))

for i, col in enumerate(non_sparse_genes[:n]):
    # Plot histogram for df_MCF7_unfiltered
    if col in df_MCF7_unfiltered.columns and col in df_HCC_unfiltered.columns:
        ax.flat[i].hist(df_MCF7_unfiltered[col], bins=30, alpha=0.5, edgecolor='black', density=True, label='MCF7')
        # Plot histogram for df2
        ax.flat[i].hist(df_HCC_unfiltered[col], bins=30, alpha=0.5, edgecolor='black', density=True, label='HCC')
        
        # KDE plots for both
        sns.kdeplot(df_MCF7_unfiltered[col], ax=ax.flat[i], bw_adjust=1, color='blue', label='MCF7 KDE')
        sns.kdeplot(df_HCC_unfiltered[col], ax=ax.flat[i], bw_adjust=1, color='red', label='HCC KDE')

        ax.flat[i].set_xlabel('')
        ax.flat[i].set_title(f"{col}\nMCF7 μ: {df_MCF7_unfiltered[col].mean():.0f}, HCC μ: {df_HCC_unfiltered[col].mean():.0f}")
        ax.flat[i].legend()

plt.suptitle("distribution of top 10 non-sparse genes in MCF7 dataset and comparison with HCC dataset", fontsize = 24)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) 
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(30, 15))

for i, col in enumerate(non_sparse_genes[:n]):
    if col in df_MCF7_unfiltered.columns and col in df_HCC_unfiltered.columns:
        combined_df = pd.DataFrame({
            'value': pd.concat([df_MCF7_unfiltered[col], df_HCC_unfiltered[col]]),
            'dataset': ['MCF7'] * len(df_MCF7_unfiltered) + ['HCC'] * len(df_HCC_unfiltered)
        })
        
        # Violin plot
        sns.violinplot( data=combined_df, x='dataset', y='value', hue='dataset',  palette=['lightblue', 'lightgreen'],  ax=ax.flat[i],  inner=None,  linewidth=1,  legend=False)
        # Boxplot overlay
        sns.boxplot(    data=combined_df, x='dataset',  y='value',  hue='dataset', palette=['lightblue', 'lightgreen'], ax=ax.flat[i],   width=0.2, showcaps=True, fliersize=2, legend=False)

        # Remove default labels
        ax.flat[i].set_xlabel('')
        ax.flat[i].set_ylabel('')
        
        # Custom title with means
        mcf7_mean = df_MCF7_unfiltered[col].mean()
        df2_mean = df_HCC_unfiltered[col].mean()
        ax.flat[i].set_title(f"{col}\nMCF7 μ: {mcf7_mean:.0f}, HCC μ: {df2_mean:.0f}")


plt.suptitle("distribution of top 20 non-sparse genes in MCF7 dataset and comparison with HCC dataset", fontsize = 16)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) 
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols = int(n/2), nrows = 2, figsize = (30,15))

for i, col in enumerate(non_sparse_genes2[:n]):
    if col in df_MCF7_unfiltered.columns and col in df_HCC_unfiltered.columns:
        # Plot histogram for df_MCF7_unfiltered
        ax.flat[i].hist(df_MCF7_unfiltered[col], bins=30, alpha=0.5, edgecolor='black', density=True, label='MCF7')
        # Plot histogram for df2
        ax.flat[i].hist(df_HCC_unfiltered[col], bins=30, alpha=0.5, edgecolor='black', density=True, label='HCC')
        
        # KDE plots for both
        sns.kdeplot(df_MCF7_unfiltered[col], ax=ax.flat[i], bw_adjust=1, color='blue', label='MCF7 KDE')
        sns.kdeplot(df_HCC_unfiltered[col], ax=ax.flat[i], bw_adjust=1, color='red', label='HCC KDE')

        ax.flat[i].set_xlabel('')
        ax.flat[i].set_title(f"{col}, MCF7 μ: {df_MCF7_unfiltered[col].mean():.0f}, HCC μ: {df_HCC_unfiltered[col].mean():.0f}")
        ax.flat[i].legend()

plt.suptitle("distribution of top 10 non-sparse genes in HCC dataset and comparison with MCF7 dataset", fontsize = 24)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) 
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(30, 15))

for i, col in enumerate(non_sparse_genes2[:n]):
    if col in df_MCF7_unfiltered.columns and col in df_HCC_unfiltered.columns:
        combined_df = pd.DataFrame({
            'value': pd.concat([df_MCF7_unfiltered[col], df_HCC_unfiltered[col]]),
            'dataset': ['MCF7'] * len(df_MCF7_unfiltered) + ['HCC'] * len(df_HCC_unfiltered)
        })
        
        # Violin plot
        sns.violinplot( data=combined_df, x='dataset', y='value', hue='dataset',  palette=['lightblue', 'lightgreen'],  ax=ax.flat[i],  inner=None,  linewidth=1,  legend=False)
        # Boxplot overlay
        sns.boxplot(    data=combined_df, x='dataset',  y='value',  hue='dataset', palette=['lightblue', 'lightgreen'], ax=ax.flat[i],   width=0.2, showcaps=True, fliersize=2, legend=False)

        # Remove default labels
        ax.flat[i].set_xlabel('')
        ax.flat[i].set_ylabel('')
        
        # Custom title with means
        mcf7_mean = df_MCF7_unfiltered[col].mean()
        df2_mean = df_HCC_unfiltered[col].mean()
        ax.flat[i].set_title(f"{col}\nMCF7 μ: {mcf7_mean:.0f}, HCC μ: {df2_mean:.0f}")


plt.suptitle("distribution of top 20 non-sparse genes in HCC dataset and comparison with MCF7 dataset", fontsize = 16)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) 
plt.show()

### SPARSITY by density of zeroes in datasets

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Choose a random subset of columns (e.g., 10 columns)
num_cols = 500
random_cols_1 = np.random.choice(df_MCF7_unfiltered.columns, size=num_cols, replace=False)
random_cols_2 = np.random.choice(df_HCC_unfiltered.columns, size=num_cols, replace=False)

# Filter both datasets with the same random columns
df1_sampled = df_MCF7_unfiltered[random_cols_1]
df2_sampled = df_HCC_unfiltered[random_cols_2]

# Compute zero masks
zero_mask1 = df1_sampled == 0
zero_mask2 = df2_sampled == 0

# Print sparsity index
sparsity1 = zero_mask1.sum().sum() / (df1_sampled.shape[0] * df1_sampled.shape[1])
sparsity2 = zero_mask2.sum().sum() / (df2_sampled.shape[0] * df2_sampled.shape[1])

print(f"Dataset MCF7 Sparsity Index: {sparsity1:.4f}")
print(f"Dataset HCC Sparsity Index: {sparsity2:.4f}")

# Plotting
plt.figure(figsize=(14, 8))

# Heatmap for Dataset 1
plt.subplot(2,1, 1)
sns.heatmap(zero_mask1, cbar=False, cmap=sns.color_palette(["white", "blue"]))
plt.title('Zero Values Heatmap - MFC7 Dataset')
plt.xlabel("Genes")
plt.ylabel("Samples")
plt.xticks([], [])
plt.yticks([], [])

# Heatmap for Dataset 2
plt.subplot(2, 1, 2)
sns.heatmap(zero_mask2, cbar=False, cmap=sns.color_palette(["white", "blue"]))
plt.title('Zero Values Heatmap - HCC Dataset')
plt.xlabel("Genes")
plt.ylabel("Samples")
plt.xticks([], [])
plt.yticks([], [])

plt.tight_layout()
plt.show()

In [ ]:
#graph of sparsity index (number of zeroes) against coutn of such index in the genes
# Calculate sparsity index for each gene (column) in both datasets
sparsity1 = ((df_MCF7_unfiltered == 0).sum(axis=0) / df_MCF7_unfiltered.shape[0])
sparsity2 = (df_HCC_unfiltered == 0).sum(axis=0) / df_HCC_unfiltered.shape[0]

# Plot overlayed histograms
plt.figure(figsize=(10, 8))

plt.hist(sparsity1, bins=80, range=(0, 1), edgecolor='black', alpha=0.6, label='MCF7 Dataset', color='blue')
plt.hist(sparsity2, bins=80, range=(0, 1), edgecolor='black', alpha=0.6, label='HCC Dataset', color='lightgreen')

plt.title("Overlayed Histogram of Gene Sparsity Indices")
plt.xlabel("Sparsity Index (Fraction of Zero Counts)")
plt.ylabel("Number of Genes")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.ylim(0,200)
plt.tight_layout()
plt.show()

The unfiltered dataset is very sparse so that it makes sense to use dimensionality reduction since the columns corresponding to some genes are almost indetically zero and only then apply clustering. \n From now on restict analysis to genes with non-zero variance

## Genes expressions

In [ ]:
# Calculate total counts per cell (row) for both datasets
df1_total_counts = df_MCF7_unfiltered.sum(axis=1)
df2_total_counts = df_HCC_unfiltered.sum(axis=1)

# Plot overlayed histograms with KDE
plt.figure(figsize=(10, 6))

sns.histplot(df1_total_counts, bins=70, kde=True, color='steelblue', label='MCF7 Dataset', alpha=0.6)
sns.histplot(df2_total_counts, bins=70, kde=True, color='darkorange', label='HCC Dataset', alpha=0.6)

plt.title("Overlayed Histogram of Total Gene Counts per Cell")
plt.xlabel("Total Counts per Cell")
plt.ylabel("Number of Samples")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
sample_names = df_MCF7_unfiltered.T.columns
cell_type = []
for name in sample_names:
    if "Norm" in name:
        cell_type.append("Norm")
    elif "Hypo" in name:
        cell_type.append("Hypo")
df_MCF7_unfiltered["CellType"] = cell_type

sample_names = df_HCC_unfiltered.T.columns
cell_type = []
for name in sample_names:
    if "Norm" in name:
        cell_type.append("Norm")
    elif "Hypo" in name:
        cell_type.append("Hypo")
df_HCC_unfiltered["CellType"] = cell_type

print("MCF7 dataset")
print(f"Checking for unbalanced classes \nNormoxia: {(df_MCF7_unfiltered["CellType"]=="Norm").sum()/ len(df_MCF7_unfiltered["CellType"]):.5}%, \t Hypoxia: {1-(df_MCF7_unfiltered["CellType"]=="Norm").sum()/ len(df_MCF7_unfiltered["CellType"]):.5}%")
print(f"Differnece in absolute count of samples with Normoxia vs (-) Hypoxia: {(df_MCF7_unfiltered["CellType"]=="Norm").sum() - (df_MCF7_unfiltered["CellType"]=="Hypo").sum()}")
print("the classes are almost perfectly balanced \n")

# Separate the features from the label
feature_columns_1 = df_MCF7_unfiltered.columns.difference(["CellType"])
# Group by the label and sum each group
grouped_sums_1 = df_MCF7_unfiltered.groupby("CellType")[feature_columns_1].sum() 
print(grouped_sums_1.head())

print("\nHCC dataset")
print(f"Checking for unbalanced classes \nNormoxia: {(df_HCC_unfiltered["CellType"]=="Norm").sum()/ len(df_HCC_unfiltered["CellType"]):.5}%, \t Hypoxia: {1-(df_HCC_unfiltered["CellType"]=="Norm").sum()/ len(df_HCC_unfiltered["CellType"]):.5}%")
print(f"Differnece in absolute count of samples with Normoxia vs (-) Hypoxia: {(df_HCC_unfiltered["CellType"]=="Norm").sum() - (df_HCC_unfiltered["CellType"]=="Hypo").sum()}")
print("the classes are almost perfectly balanced \n")

# Separate the features from the label
feature_columns_2 = df_HCC_unfiltered.columns.difference(["CellType"])
# Group by the label and sum each group
grouped_sums_2 = df_HCC_unfiltered.groupby("CellType")[feature_columns_2].sum() 
grouped_sums_2.head()

In [ ]:
top_n = 20

# Create subplots with 2 rows and 1 column
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# MCF7 dataset
total_sums_1 = grouped_sums_1.sum(axis=0)

# Sort columns by total sum and pick top N
top_columns_1 = total_sums_1.sort_values(ascending=False).head(top_n).index
top_data_1 = grouped_sums_1[top_columns_1]

# First subplot: MCF7 dataset
top_data_1.T.plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_title("Genes' Expression by Cell Condition - MCF7 Dataset", fontsize=18)
axes[0].set_xlabel('Genes')
axes[0].set_ylabel('Total Counts')
axes[0].legend(title='Cell Condition')
axes[0].grid(axis='y', linestyle='--', alpha=0.5)

# HCC dataset
total_sums_2 = grouped_sums_2.sum(axis=0)

# Sort columns by total sum and pick top N
top_columns_2 = total_sums_2.sort_values(ascending=False).head(top_n).index
top_data_2 = grouped_sums_2[top_columns_2]

# Second subplot: HCC dataset
top_data_2.T.plot(kind='bar', stacked=True, ax=axes[1])
axes[1].set_title("Genes' Expression by Cell Condition - HCC Dataset", fontsize=18)
axes[1].set_xlabel('Genes')
axes[1].set_ylabel('Total Counts')
axes[1].legend(title='Cell Condition')
axes[1].grid(axis='y', linestyle='--', alpha=0.5)

# Final layout adjustment
plt.tight_layout()
plt.show()

In [ ]:
# 2. Get top genes from dataset 1
total_sums = grouped_sums_1.sum(axis=0)
top_genes = total_sums.sort_values(ascending=False).head(top_n).index
top_HCC = set(top_genes) & set(df_HCC_unfiltered.columns)
top_MCF7 = set(top_genes) & set(df_MCF7_unfiltered.columns)
top_genes = list(top_HCC & top_MCF7)

# 3. Subset both datasets
df1_top = grouped_sums_1[top_genes].copy()
df2_top = grouped_sums_2[top_genes].copy()

# 4. Add 'Condition' column before melting
df1_top['Condition'] = df1_top.index
df2_top['Condition'] = df2_top.index

# 5. Melt both to long format
df1_long = df1_top.melt(id_vars='Condition', var_name='Gene', value_name='Count')
df1_long['Dataset'] = 'MCF7 Dataset'
df2_long = df2_top.melt(id_vars='Condition', var_name='Gene', value_name='Count')
df2_long['Dataset'] = 'HCC Dataset'

# 6. Combine both
combined_long = pd.concat([df1_long, df2_long], ignore_index=True)

# 7. Create composite category: Gene + Condition
combined_long['Gene_Condition'] = combined_long['Gene'] + ' (' + combined_long['Condition'] + ')'

# 8. Plot side-by-side bars
plt.figure(figsize=(16, 6))
sns.barplot(
    data=combined_long,
    x='Gene_Condition',
    y='Count',
    hue='Dataset',
    palette=['blue', 'red']
)

# 9. Styling
plt.title("Gene Expression by Condition — HCC Dataset vs MCF7 Dataset", fontsize=16)
plt.xlabel("Gene (Condition)", fontsize=12)
plt.ylabel("Total Counts", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Dataset')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
df_HCC_unfiltered.drop(columns="CellType", inplace=True)
df_MCF7_unfiltered.drop(columns="CellType", inplace=True)

This suggests that some genes are more important than others to identify hypoxic cells since some are linked to a particular condition. This means that feature selection and dimensionality reduction might be very helpfuul to remove genes produced for both hypoxic and normal cells

## Evaluating correlation of samples and genes 

In [ ]:
n_genes = 50
top_genes_1 = df_MCF7_unfiltered.iloc[:, :-2].sum().sort_values(ascending=False).head(n_genes).index
sampled_cells_1 = df_MCF7_unfiltered.sample(25)
truncated_rows_1 = [i[12:27] for i in sampled_cells_1.index.tolist()]

top_genes_2 = df_HCC_unfiltered.iloc[:, :-2].sum().sort_values(ascending=False).head(n_genes).index
sampled_cells_2 = df_HCC_unfiltered.sample(25)
truncated_rows_2 = [i[12:38] for i in sampled_cells_2.index.tolist()]

# Plotting
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Heatmap for Dataset 1 (MCF7)
sns.heatmap(sampled_cells_1[top_genes_1], ax=axes[0], cbar=True, cmap="coolwarm")
axes[0].set_title('Expression Heatmap: Top 20 Genes in Sampled Cells - MCF7 Dataset')
axes[0].set_xlabel("Genes")
axes[0].set_ylabel("Sample Cells")
axes[0].set_yticklabels(truncated_rows_1, rotation=0)

# Heatmap for Dataset 2 (HCC)
sns.heatmap(sampled_cells_2[top_genes_2], ax=axes[1], cbar=True, cmap="coolwarm")
axes[1].set_title('Expression Heatmap: Top 20 Genes in Sampled Cells - HCC Dataset')
axes[1].set_xlabel("Genes")
axes[1].set_ylabel("Sample Cells")
axes[1].set_yticklabels(truncated_rows_2, rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Heatmap for Dataset 1 (MCF7)
sns.heatmap(sampled_cells_1[top_genes_1].corr(), ax=axes[0], cbar=True, cmap="coolwarm")
axes[0].set_title('Gene correlation matrix: Top 20 Genes in Sampled Cells - MCF7 Dataset')
axes[0].set_xlabel("Genes")
axes[0].set_ylabel("Genes")

# Heatmap for Dataset 2 (HCC)
sns.heatmap(sampled_cells_2[top_genes_2].corr(), ax=axes[1], cbar=True, cmap="coolwarm")
axes[1].set_title('Gene correlation matrix: Top 20 Genes in Sampled Cells - HCC Dataset')
axes[1].set_xlabel("Genes")
axes[1].set_ylabel("Genes")

plt.tight_layout()
plt.show()

In [ ]:
# Scale and compute distances for MCF7                                               ######################### CHECK BETTER AND DECIDE WHETHER TO KEEP HERE ##############################
scaler = StandardScaler()
df_scaled_1 = scaler.fit_transform(df_MCF7_unfiltered[top_genes_1].T)
distance_matrix_1 = pdist(df_scaled_1, metric='correlation')
distance_square_1 = squareform(distance_matrix_1)

# Scale and compute distances for HCC
df_scaled_2 = scaler.fit_transform(df_HCC_unfiltered[top_genes_2].T)
distance_matrix_2 = pdist(df_scaled_2, metric='correlation')
distance_square_2 = squareform(distance_matrix_2)

# Create two clustermaps separately
g1 = sns.clustermap(distance_square_1, cmap='coolwarm', xticklabels=top_genes_1, yticklabels=top_genes_1)
g1.cax.set_position([.99, .08, .03, .74])
g2 = sns.clustermap(distance_square_2, cmap='coolwarm', xticklabels=top_genes_2, yticklabels=top_genes_2)
g2.cax.set_position([.99, .08, .03, .74])

# Optional: improve layout
g1.fig.suptitle('Gene Correlation Matrix - MCF7 Dataset (Top 20 Genes)', y=1.05, fontsize=16)
g2.fig.suptitle('Gene Correlation Matrix - HCC Dataset (Top 20 Genes)', y=1.05, fontsize=16)

plt.show()

In [ ]:
# Sample 100 cells
sampled_cells_1 = df_MCF7_unfiltered.sample(100)
sampled_cells_2 = df_HCC_unfiltered.sample(100)

# Select top 20 genes by total expression
top_genes_1 = df_MCF7_unfiltered.iloc[:, :-2].sum().sort_values(ascending=False).head(20).index
top_genes_2 = df_HCC_unfiltered.iloc[:, :-2].sum().sort_values(ascending=False).head(20).index

# Extract expression data for top genes
sampled_data_1 = sampled_cells_1[top_genes_1]
sampled_data_2 = sampled_cells_2[top_genes_2]

# Compute sample-to-sample correlation
sample_corr_1 = sampled_data_1.corr(method='pearson') if sampled_data_1.shape[0] < sampled_data_1.shape[1] else sampled_data_1.T.corr()
sample_corr_2 = sampled_data_2.corr(method='pearson') if sampled_data_2.shape[0] < sampled_data_2.shape[1] else sampled_data_2.T.corr()

# Truncate sample names
truncated_names_1 = [idx[12:27] for idx in sample_corr_1.index]
truncated_names_2 = [idx[12:38] for idx in sample_corr_2.index]

# Apply truncated names to both axes
sample_corr_1.index = truncated_names_1
sample_corr_1.columns = truncated_names_1
sample_corr_2.index = truncated_names_2
sample_corr_2.columns = truncated_names_2

# Plotting
fig, axes = plt.subplots(2, 1, figsize=(14, 14))

# MCF7 correlation heatmap
sns.heatmap(sample_corr_1, ax=axes[0], cmap="coolwarm", cbar=True)
axes[0].set_title("Sample Correlation Heatmap - MCF7 (Top 20 Genes)")
axes[0].set_xlabel("Samples")
axes[0].set_ylabel("Samples")

# HCC correlation heatmap
sns.heatmap(sample_corr_2, ax=axes[1], cmap="coolwarm", cbar=True)
axes[1].set_title("Sample Correlation Heatmap - HCC (Top 20 Genes)")
axes[1].set_xlabel("Samples")
axes[1].set_ylabel("Samples")

plt.tight_layout()
plt.show()

Samples are not really independent as they should be, as can be seen from bright heatmap colour

## BONUS: OUTLIER DETECTION WITH ISOLATION FOREST on MCF7 dataset

As can be seen below, traditional statistical tests relying on IQR and zscore classify too many points as outliers due to high-dimensionality and sparsity issues. Therefore, we try to implement a more complex model for outlier detection based on random forests, called isolation forest as implemented by scikit-learn.\
This ML algorithm, ISOLATION FOREST, trains is an ensemble of trees for which at each step, a selection of a random feature is performed and then a split value is randomly selected leading to a recusrive partitioning of the feature space, which ends when all samples occupy a different leaf, all leaves are pure. The number of such splits required to arrive (from the first split) to a pure leaf is a measure of normality of the sample and thus used as the model's decision function. This happens because, citing scikit documentation, "Random partitioning produces noticeably shorter paths for anomalies. Hence, when a forest of random trees collectively produce shorter path lengths for particular samples, they are highly likely to be anomalies."

In [ ]:
# Z-score method
from scipy.stats import zscore
z_scores = np.abs(zscore(df_MCF7_unfiltered))
df_no_outliers = df_MCF7_unfiltered[(z_scores < 3).all(axis=1)]
print(f"Removed {df_MCF7_unfiltered.shape[0] - df_no_outliers.shape[0]} outliers using Z-score method.")

#IQR method
Q1 = df_MCF7_unfiltered.quantile(0.25)
Q3 = df_MCF7_unfiltered.quantile(0.75)
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Keep rows within bounds
df_iqr_no_outliers = df_MCF7_unfiltered[~((df_MCF7_unfiltered < lower_bound) | (df_MCF7_unfiltered > upper_bound)).any(axis=1)]
print(f"Removed {df_MCF7_unfiltered.shape[0] - df_iqr_no_outliers.shape[0]} outliers using IQR method.")

In [ ]:
# Filter out samples with more than 90% zeros
sparsity_threshold = 0.90
non_zero_counts = (df_HCC_unfiltered > 0).sum(axis=1)
sample_sparsity = 1 - (non_zero_counts / df_HCC_unfiltered.shape[1])
print(sample_sparsity)
filtered_samples = df_HCC_unfiltered[sample_sparsity <= sparsity_threshold]

# Show how many samples were retained
print("Samples retained:", filtered_samples.shape[0])
print("Samples removed:", df_HCC_unfiltered.shape[0] - filtered_samples.shape[0])

In [ ]:
print(np.mean(sample_sparsity)-np.std(sample_sparsity))

In [ ]:
# Filter out samples with more than 90% zeros
sparsity_threshold = 0.90
non_zero_counts = (df_HCC_unfiltered.T > 0).sum(axis=1)
genes_sparsity = 1 - (non_zero_counts / df_HCC_unfiltered.T.shape[1])
print(sample_sparsity)
filtered_genes = df_HCC_unfiltered.T[genes_sparsity <= sparsity_threshold]

# Show how many samples were retained
print("Samples retained:", filtered_genes.shape[0])
print("Samples removed:", df_HCC_unfiltered.T.shape[0] - filtered_genes.shape[0])

In [ ]:
print(np.mean(sample_sparsity)-np.std(sample_sparsity))

Since the distributions are very much skewed and dataset is very sparse, traditional quartiles and disitribution tests lead to almost all datapoints being considered as outliers 

In what follows, we first apply standrdization to the featured and then apply PCA to the MCF7 (easier) dataset to evalaute how far different data representations can influence the findings of siolation forest. We train the Isolation Forest algorithm using 100 components and visualise the results in 2d.

In [ ]:
import umap.umap_ as umap
#data processing
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_MCF7_unfiltered)

# Reduce to 2D with PCA for visualization
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(df_scaled)

#pca for data compression and outlier detection
pca_out = PCA(n_components=2500, random_state=42)
pca_result_out = pca.fit_transform(df_scaled)

In [ ]:
# Apply Isolation Forest to data scaled 
iso_forest = IsolationForest(n_estimators=100, max_features=1, max_samples=100, contamination="auto", n_jobs=10, random_state=4)
outlier_labels_std = iso_forest.fit_predict(df_scaled) #fit the model using only the standardized data

# Add results back to a DataFrame
df_result = pd.DataFrame(df_scaled, columns=df_MCF7_unfiltered.columns)
df_result['outlier'] = outlier_labels_std == -1  # True if outlier
df_result['PC1'] = pca_result[:, 0]
df_result['PC2'] = pca_result[:, 1]

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(df_result.loc[~df_result['outlier'], 'PC1'],
            df_result.loc[~df_result['outlier'], 'PC2'],
            c='blue', label='Inliers', alpha=0.6)
plt.scatter(df_result.loc[df_result['outlier'], 'PC1'],
            df_result.loc[df_result['outlier'], 'PC2'],
            c='red', label='Outliers', alpha=0.8)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Outlier Detection in MCF7 standardized Data (Isolation Forest)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Apply Isolation Forest to principal components
iso_forest = IsolationForest(n_estimators=100, max_features=1, max_samples=100, contamination="auto", n_jobs=10, random_state=4)
outlier_labels = iso_forest.fit_predict(pca_result_out)

# Add results back to a DataFrame
df_result = pd.DataFrame(df_scaled, columns=df_MCF7_unfiltered.columns)
df_result['outlier'] = outlier_labels == -1  # True if outlier
df_result['PC1'] = pca_result[:, 0]
df_result['PC2'] = pca_result[:, 1]

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(df_result.loc[~df_result['outlier'], 'PC1'],
            df_result.loc[~df_result['outlier'], 'PC2'],
            c='blue', label='Inliers', alpha=0.6)
plt.scatter(df_result.loc[df_result['outlier'], 'PC1'],
            df_result.loc[df_result['outlier'], 'PC2'],
            c='red', label='Outliers', alpha=0.8)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Outlier Detection (Isolation Forest) in MCF7 Data after PCA with 200 comps')
plt.legend()
plt.xlim(-100, 130)
plt.grid(True)
plt.show()

Using the contaimnation = "auto" parameter as is usually done in unsupervised analysis, gives very poor results classifying most of the points as outliers or none of the points probably because of the same sparsity and scaling issues discussed above. Moreover, the results are not consistent since they depend heavily on the data representation as can be seen from teh fact that the dataset after PCA a majority of points considered outliers and before it, instead no point is deemed to be an outlier.Therefore, we suspect method not be highly reliable for our dataset

In [ ]:
# Parameters to explore
contamination_values = [0.01, 0.05, "auto"] #although the analysis is unsupervised and auto is set explcitly to 0.01,0.05, it is done just to see how the algorithm works and how much
                                            # this affects the outliers predicted. Answer: dependence is too significant and so no really reliable for blind detection 
max_samples_values = [0.4, 0.5, 0.75, 'auto']

# Set up subplot grid
fig, axes = plt.subplots(len(contamination_values), len(max_samples_values), figsize=(18, 12))
fig.suptitle('Isolation Forest Outlier Detection: Various Contamination & Max_Samples', fontsize=16)

for i, contamination in enumerate(contamination_values):
    for j, max_samples in enumerate(max_samples_values):
        # Fit Isolation Forest
        iso_forest = IsolationForest(
            n_estimators=100,
            max_features=1,
            max_samples=max_samples,
            contamination=contamination,
            n_jobs=-1,
            random_state=4
        )
        outlier_labels = iso_forest.fit_predict(pca_result_out)
        outliers = outlier_labels == -1
        
        # Prepare DataFrame for plotting
        df_result = pd.DataFrame({
            'PC1': pca_result[:, 0],
            'PC2': pca_result[:, 1],
            'outlier': outliers
        })
        
        ax = axes[i, j]
        ax.scatter(df_result.loc[~df_result['outlier'], 'PC1'],
                   df_result.loc[~df_result['outlier'], 'PC2'],
                   c='blue', label='Inliers', alpha=0.5)
        ax.scatter(df_result.loc[df_result['outlier'], 'PC1'],
                   df_result.loc[df_result['outlier'], 'PC2'],
                   c='red', label='Outliers', alpha=0.8)
        ax.set_title(f"Contam: {contamination}, MaxSamples: {max_samples}")
        ax.set_xlim(-100, 130)
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        ax.grid(True)

# Only show legend in one plot
for i in range(len(contamination_values)):
    for j in range(len(max_samples_values)):
        axes[i,j].legend()

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
#we try plotting the deciision function for isolation forest to gain more intel on the model's behaviour
clf = IsolationForest(max_samples="auto", contamination="auto", n_estimators=100, max_features=1, random_state=4) 
clf.fit(pca_result[:,:2])
y = clf.predict(pca_result[:,:2])

fig, ax = plt.subplots(figsize=(10,10)) 

disp = DecisionBoundaryDisplay.from_estimator(
    clf,
    pca_result[:,:2],
    response_method="decision_function",
    alpha=0.5,
    cmap=plt.cm.coolwarm,
    ax =ax
)
colors = np.where(y == -1, "blue", "red")
scatter = disp.ax_.scatter(pca_result[:, 0], pca_result[:, 1], c=colors, s=20, edgecolor="k")

# Plot formatting
# Title and labels
ax.set_title("Isolation Forest Decision Boundary on PCA-Reduced Data")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_aspect('equal')

# Create legend manually
import matplotlib.patches as mpatches
legend_handles = [
    mpatches.Patch(color='red', label='Inliers'),
    mpatches.Patch(color='blue', label='Outliers')
]
ax.legend(handles=legend_handles, title="Predicted Class")

# Add colorbar correctly
cb = plt.colorbar(disp.ax_.collections[0], ax=ax, shrink=0.6)
cb.set_label('Decision Function Value')
plt.tight_layout()
plt.show()

In [ ]:
print(f"{sum(y==1)} out of {len(y)}")

In [ ]:
sample_names = df_HCC_unfiltered.T.columns
conditions = []
for name in sample_names:
    if "Norm" in name:
        conditions.append("Norm")
    elif "Hypo" in name:
        conditions.append("Hypo")
    else:
        conditions.append("Unknown")

colors_HCC = ["blue" if c == "Norm" else "red" if c == "Hypo" else "gray" for c in conditions]
condition_labels_HCC = np.array([0 if c == "Norm" else 1 if c == "Hypo" else 2 for c in conditions])

sample_names = df_MCF7_unfiltered.T.columns
conditions = []
for name in sample_names:
    if "Norm" in name:
        conditions.append("Norm")
    elif "Hypo" in name:
        conditions.append("Hypo")
    else:
        conditions.append("Unknown")

colors_MCF7 = ["blue" if c == "Norm" else "red" if c == "Hypo" else "gray" for c in conditions]
condition_labels_MCF7 = np.array([0 if c == "Norm" else 1 if c == "Hypo" else 2 for c in conditions])

In [ ]:
print(condition_labels_MCF7[y==1])

In [ ]:
def extract_labels(df):
    sample_names = df.T.columns
    conditions = []
    for name in sample_names:
        if "Norm" in str(name):
            conditions.append(0)
        elif "Hypo" in str(name):
            conditions.append(1)

    colors = ["blue" if c == 0 else "red" for c in conditions]
    return conditions, colors 
conditions, colors = extract_labels(df_HCC_unfiltered)

In [ ]:
# hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from scipy.stats import uniform
from scipy.stats import loguniform

estimator = MLPClassifier()

# define parameter distribution
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (200,), (50, 50), (100, 100), (200, 200)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': loguniform(1e-5, 1),
    'batch_size': [None, 10, 20, 50],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [200, 500, 1000]
}

# search
random_search = RandomizedSearchCV(
    estimator=MLPClassifier(),
    param_distributions=param_dist,
    cv=5,
    n_iter=50,
    random_state=42,
    n_jobs=-1
)

# HCC
random_search.fit(df_HCC_unfiltered, conditions)

# best model HCC
best_mlp_HCC = random_search.best_estimator_
best_params_HCC = random_search.best_params_
best_score_HCC = random_search.best_score_

# # MCF7
# random_search.fit(X_train_MCF7, y_train_MCF7)
# # best model MCF7
# best_mlp_MCF7 = random_search.best_estimator_
# best_params_MCF7 = random_search.best_params_
# best_score_MCF7 = random_search.best_score_



As can be seen from above analysis, the results of Isolation Forest appear to be qualitatively/graphically sensible given the plots obtained and provide a good way to detect outliers, when in possession of estimates on the share of outliers in the dataset.
However, the outliers found by the algorithm very much depend on the parmeters used, especially on the contamination parameter, (literally the approximate proportion of outliers) which requires previous knowledge of outlier presence or of anomalies because of some bilogical reason. Since there is no good reason to suppose/assume that any specfic quantity of anomalies were produced in the data, this method does not provide enough evidence for removing or handling differently some datapoints.\
Moreover, Isolation Forest algorithm is also very much dependent upon the data representation since it partitions the feature space. Therefore, if we had applied it to the UMAP or TSNE mebedding, using the same or a similar number of components, we would have gotten completely different results.\
FInally, the method is discarded for future analysis because of the above reasons and the presence of better alternatives

Instead, we will attempt to identify samples corresponding to noise or outliers in the unsupervised analysis file by resorting to more general techniques, namely silhouette score graphs and DBSCAN.